In [1]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import config
from parse_functions import *
# ссылка на основной файл гугл докс (на каждом лите отдельный клиент со ссылками на файлы с данными)
base_link = config.base_link


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [3]:
# функция, которая на входе принимает 1 строку из основного датаФрейма
# строка содержит название отчета и ссылку на гугл докс с данными
# name - название клиента
# report - название отчета
def get_parse_reports(df, name, report):
    # т.к. в одном гугл доксе может быть много отчетов на разных листах, мы создаем словарь
    tmp_dict = {}
    # забираем ссылку на гугл докс, из нее получаем ИД и забираем данные 
    content = get_data_from_sheet(df['report_link'][0])
    # проверяем - если есть доступк к гугл доксу, то запускаем парсинг данных
    # но может прийти ошибка, если НЕТ общего доступа
    try:
        # забираем названия листов в гугл доксе
        target_sheet_names = pd.ExcelFile(content)
        # проходим через цикл по названиям всех листов
        for target_sheet in target_sheet_names.sheet_names:
            # если в названиии листа есть слово чек-лист, то вызываем функцию для парсинга данных
            if 'чек-лист' in target_sheet.lower():
                # final_df = parse_check_list_report(content, target_sheet, name)
                # для отчетов калиниград ключом является название отчета из основного файла
                if 'калининград' in name.lower():
                    tmp_dict[report] = parse_check_list_report(content, target_sheet, name, report)
                else:
                # для всех остальных ключ - это название листа
                    tmp_name = str(target_sheet).lower()
                    print(tmp_name)
                    tmp_dict[tmp_name] = parse_check_list_report(content, target_sheet, name, report)
            if 'срм' in report.lower():
                tmp_dict[report] = parse_crm_report(content, target_sheet, name, report)
        return tmp_dict
    # если доступа нет, то возвращаем инфо -к какому отчету НЕ получилось подключиться
    except:
        print(f'Нет доступа к файлу: Клиента - {name} / Отчет: {report}')

In [4]:
# забираем данные из основной управленческой таблицы
content = get_data_from_sheet(base_link)
sheet_names = pd.ExcelFile(content)

In [5]:
sheet_names.sheet_names

['Калининград МореАвто(ЮтоКарс)', 'Екатеринбург АтлантикПРО', 'АвтоБелогорье']

In [31]:
# создаем словарь, в который сохраним результаты парсинга всех отчетов
tmp_reports_dict = {}
# проходим через цикл по всем листам в управленческой таблице
for name in sheet_names.sheet_names:
    print(name)
    
    cols_range = 'A:B' # задаем диапазон полей, которые нам нужны
    df = pd.read_excel(content, sheet_name=name, usecols=cols_range, names=['report_name', 'report_link'])
    # нормализуем назания отчетов - приводим в нижний регистр, обрезаем по краям, избаляемся от технических символов(перенос строки и тд)
    df['report_name'] = df['report_name'].str.lower().str.strip().str.replace('\n', ' ')
    # создаем отдельный список с названиями отчетов и проодим по нему через цикл
    reports_list = list(df['report_name'])
    for report in reports_list:
        parse_info = df[df['report_name']==report].reset_index(drop=True)
    # запускаем блок парсинга отчетов с основными данными - Чек листы
        if 'чек' in report.lower():
            if 'калининград' in name.lower():
                print(report)
                tmp_reports_dict[report] = get_parse_reports(parse_info, name, report) 
            else:
                tmp_reports_dict[name] = get_parse_reports(parse_info, name, report)
        # запускаем блок парсинга отчетов с данными CRM
        if 'срм' in report.lower():
                print(report)
         # создаем спец. ключ для словаря
                key = name + '_' + report
                tmp_reports_dict[key] = get_parse_reports(parse_info, name, report)

Калининград МореАвто(ЮтоКарс)
чек-лист калининград ап
чек-лист калининград оп
чек-лист калининград выкуп
чек-лист калининград парсинг
срм ап
срм оп
Екатеринбург АтлантикПРО
чек-лист ап
чек-лист чаты-звонки
чек-лист комиссия
чек-лист чаты
срм чаты_звонки
срм ап
АвтоБелогорье
чек-лист оп exeed белгород
чек-лист оп faw_jac белгород
чек-лист оп omoda_jaecoo старый
чек-лист оп jetour старый оскол
чек-лист оп лада белгород
чек-лист ап
срм лада
срм джетур
срм омода
срм фав
Нет доступа к файлу: Клиента - АвтоБелогорье / Отчет: срм фав
срм эксид


In [32]:
tmp_reports_dict['Екатеринбург АтлантикПРО_срм ап']['срм ап'].head()

,дата,команда,кол-во лидов,кол-во согласившихся на визит,кол-во состоявшихся визитов,client,dashboard
0,2024-07-01,"Репина, 13",,,,Екатеринбург АтлантикПРО,срм ап
1,2024-08-07,"Таганская, 77",6,,2,Екатеринбург АтлантикПРО,срм ап
2,2024-08-07,"Репина, 13",,,,Екатеринбург АтлантикПРО,срм ап
3,2024-08-08,"Таганская, 77",12,,3,Екатеринбург АтлантикПРО,срм ап
4,2024-08-08,"Репина, 13",,,,Екатеринбург АтлантикПРО,срм ап


In [33]:
tmp_reports_dict['чек-лист калининград выкуп']['чек-лист калининград выкуп'].head(3)

,номер события,ссылка на звонок,источник контакта,дата звонка,фио менеджера,менеджер представился по имени,менеджер назвал компанию,менеджер выявил причину продажи,менеджер задал 2 вопроса и более по авто,озвучен оффер на кэф,менеджер спросил мнение клиента о предложении,фиксация возражения клиента,менеджер аргументирует возражение клиента,менеджер предложил визита для точной оценки,менеджер договорился о следующем шаге с клиентом,оценка звонка,клиент согласился на визит,по сделке стоит актуальная задача,"причины, почему не согласился на визит",желаемая цена продажи для клиента,комментарии,статус,client,dashboard
1,,,,,Вес вопроса,0.04,0.04,0.06,0.06,0.25,0.15,да (какое?)/нет,0.15,0.15,0.1,1,да/нет,да/нет,,,,,Калининград МореАвто(ЮтоКарс),чек-лист калининград выкуп
0,АМ00000440,https://disk.yandex.ru/d/kOU3ZC61l1Cg9Q,Авито,2024-01-06,Анна Гзирян2,0.04,0.04,0,0,0,0,за сколько готовы купить?,0.15,0.15,0.1,0.48,да,да,,,,Одобрен,Калининград МореАвто(ЮтоКарс),чек-лист калининград выкуп
1,АМ00000431,https://disk.yandex.ru/d/TK72P4DR2wQa0A,Авито,2024-01-06,Анна Гзирян2,0.04,0.04,0,0,0,0,Вы частное лицо?,0,0,0,0.08,нет,,"Не стал разговаривать , узнав, что звонок из автосалона",,,Одобрен,Калининград МореАвто(ЮтоКарс),чек-лист калининград выкуп


In [34]:
tmp_reports_dict['АвтоБелогорье']['чек-лист оп faw_jac белгород'].head(3)

,номер события,ссылка на звонок,источник контакта,дата звонка,фио менеджера,менеджер представился по имени,менеджер назвал компанию,менеджер уточнил сроки покупки,менеджер уточнил знаком ли клиент с авто или нет,менеджер предложил traid-in( с выгодой),менеджер предложил кредит(с выгодой),"менеджер создал срочность (много интересуются, акция)",программирование,менеджер спросил мнение клиента о предложении,фиксация возражения клиента,менеджер аргументирует возражение клиента,менеджер предложил визит в салон (с объяснением для чего),менеджер договорился о следующем шаге с клиентом,оценка звонка,клиент согласился на визит,по сделке стоит актуальная задача,марка авто,"причины, почему не согласился на визит",комментарии,статус,команда,сценарий соблюден,client,dashboard
1,,,,,Вес вопроса,0.01,0.01,0.05,0.12,0.05,0.12,0.06,0.06,0.12,да (какое?)/нет,0.12,0.16,0.12,1,да/нет,да/нет,,,,,,да/нет,АвтоБелогорье,чек-лист оп faw_jac белгород
0,344598,https://drive.google.com/file/d/1RcTFE2mtlLQiviuuQVuSSyRQ4o0kWLtX/view?usp=drive_link,другое,2024-08-29,Калинин Руслан,0,0,0,0,0,0,0,0,0,нет,0,0,0,0,нет,да,УАЗ Patriot,"1-я линия оператор, менеджер перезвонит","1-я линия оператор, менеджер перезвонит",Удалить,,,АвтоБелогорье,чек-лист оп faw_jac белгород
1,345026,https://drive.google.com/file/d/1cE4thhkdQoolVIfHzGbe2N9IDW4_TKwL/view?usp=drive_link,другое,2024-08-30,Калинин Руслан,0,0,0,0,0,0,0,0,0,нет,0,0,0,0,нет,да,JAC T8,"1-я линия оператор, менеджер перезвонит","1-я линия оператор, менеджер перезвонит",Одобрен,,,АвтоБелогорье,чек-лист оп faw_jac белгород


In [35]:
tmp_reports_dict['Екатеринбург АтлантикПРО']['чек-лист чаты-звонки'].head(3)

,ссылка на рабочий лист,ссылка на звонок,автосалон,дата звонка,марка авто,фио менеджера,менеджер представился по имени,менеджер назвал компанию,менеджер уточнил сроки покупки,менеджер предложил traid-in( с выгодой),менеджер предложил кредит(с выгодой),"менеджер создал срочность (много интересуются, акция)",предложил альтернативу или рассказал о широком ассортименте авто,формат ответа на вопрос о цене,менеджер спросил мнение клиента о предложении,фиксация возражения клиента,менеджер аргументирует возражение клиента,менеджер предложил визит в салон сегодня,менеджер договорился о следующем шаге с клиентом,менеджер рассказал как добраться до салона,менеджер уточнил контактный номер телефона?,менеджер предложил отправить визитку?,оценка звонка,клиент согласился на визит,по сделке стоит актуальная задача,"причины, почему не согласился на визит",комментарии,статус,client,dashboard
3,,,,,,Вес вопроса,0.01,0.01,0.05,0.05,0.05,0.08,0.06,0.1,0.1,да (какое?)/нет,0.1,0.14,0.1,0.05,0.05,0.05,1,да/нет,да/нет,,,,Екатеринбург АтлантикПРО,чек-лист чаты-звонки
0,https://atlantikpro.autocrm.ru/yii/sale/case/view/103015/,https://api.selcdn.ru/v1/SEL_56486/autocrm-1/atlantikpro/2024/07/01/ad8960639105d5d2a66476a68802d7fa18f3f432.mp3,"Таганская, 77",2024-07-01,Chevrolet Niva,Мальцева Лидия,0.01,0.01,0,0,0,0,0,0,0,,0,0,0,0,0,0,0.02,нет,нет,,повторный звонок - узнать планы клиента,Удалить,Екатеринбург АтлантикПРО,чек-лист чаты-звонки
1,https://atlantikpro.autocrm.ru/yii/sale/case/view/103018/,https://api.selcdn.ru/v1/SEL_56486/autocrm-1/atlantikpro/2024/07/01/cb66e5e97190d2ff9f6bfd820f9491042dd5c30b.mp3,"Таганская, 77",2024-07-01,3009 A1,Мальцева Лидия,0.01,0.01,0.05,0.05,0.05,0.08,0.06,0.1,0.1,,0.1,0.14,0.1,0.05,0.05,0.05,1,да,нет,,клиент сам инициировал визит,Одобрен,Екатеринбург АтлантикПРО,чек-лист чаты-звонки


In [ ]:
sheet_names.sheet_names

In [ ]:
for name in sheet_names.sheet_names:
   
    if 'белогорье' in name.lower():
        print(name)
        cols_range = 'A:B' # задаем диапазон полей, которые нам нужны
        df = pd.read_excel(content, sheet_name=name, usecols=cols_range, names=['report_name', 'report_link'])
        # нормализуем назания отчетов - приводим в нижний регистр, обрезаем по краям, избаляемся от технических символов(перенос строки и тд)
        df['report_name'] = df['report_name'].str.lower().str.strip().str.replace('\n', ' ')
        # создаем отдельный список с названиями отчетов и проодим по нему через цикл
        reports_list = list(df['report_name'])
        for report in reports_list:
            if 'срм эксид' in report.lower():
                print(report)
                parse_info = df[df['report_name']==report].reset_index(drop=True)

In [ ]:
parse_info

In [ ]:
google_link = parse_info['report_link'][0]

In [ ]:
start_index = str(google_link).find('/d/')
end_index = str(google_link).find('/edit?')
spreadsheetId = google_link[start_index+3:end_index]

In [ ]:
 # забираем ссылку на гугл докс, из нее получаем ИД и забираем данные 
content = get_data_from_sheet(parse_info['report_link'][0])
# забираем названия листов в гугл доксе
target_sheet_names = pd.ExcelFile(content)

In [ ]:
for target_sheet in target_sheet_names.sheet_names:
    # если в названиии листа есть слово чек-лист, то вызываем функцию для парсинга данных
    if 'показатели' in target_sheet.lower():
        print(target_sheet)

In [ ]:
df_tmp = pd.read_excel(content, sheet_name=target_sheet, header=None)
df_tmp = df_tmp.fillna('')
df_tmp.columns = df_tmp.iloc[0].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
df_tmp = df_tmp.iloc[1:]
df_tmp = df_tmp[df_tmp['дата']!='']
df_tmp['дата'] = pd.to_datetime(df_tmp['дата']).dt.date # приводим в формат даты
df_tmp = df_tmp.reset_index(drop=True)

In [ ]:
df_tmp

In [ ]:
df_tmp = pd.read_excel(content, sheet_name=target_sheet)
df_tmp = df_tmp.fillna('')
df_tmp.columns = df_tmp.iloc[0].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
weight_index = list(df_tmp[df_tmp['фио менеджера'].str.lower()=='вес вопроса'].index)[0] # берем индекс окончания таблицы с данными
df_weight = df_tmp.iloc[weight_index:weight_index+1]
# df_tmp = df_tmp[df_tmp['дата звонка']!='']

In [ ]:

    content = get_data_from_sheet(df['report_link'][0])
    target_sheet_names = pd.ExcelFile(content)
    for target_sheet in target_sheet_names.sheet_names:
        if 'Чек-лист' in target_sheet:
            # final_df = parse_check_list_report(content, target_sheet, name)
            tmp_dict[target_sheet] = parse_check_list_report(content, target_sheet, name)

In [ ]:
df_tmp.head()

In [ ]:
for target_sheet in target_sheet_names.sheet_names[:1]:
    print(target_sheet)
    if 'Чек-лист' in target_sheet:
        df_tmp = pd.read_excel(content, sheet_name=target_sheet)
        df_tmp = df_tmp.fillna('')
        df_tmp.columns = df_tmp.iloc[0].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
        weight_index = list(df_tmp[df_tmp['фио менеджера'].str.lower()=='вес вопроса'].index)[0] # берем индекс окончания таблицы с данными
        start_index = weight_index + 1
        df_weight = df_tmp.iloc[weight_index:start_index]
        # df_weight = df_tmp.iloc[3:4]
        df_tmp = df_tmp[df_tmp['дата звонка']!='']
        df_tmp = df_tmp.iloc[start_index:]
        df_tmp['дата звонка'] = pd.to_datetime(df_tmp['дата звонка']).dt.date # приводим в формат даты
        df_tmp = df_tmp.reset_index(drop=True)
        final_df = pd.concat([df_weight, df_tmp])
        final_df['client'] = name
        final_df['dashboard'] = report
